# iNaturalist status updates by state - QLD

Using the file produced in the collate-status-taxa.ipynb: `inat-aust-status-taxa.csv`, generate lists to update iNaturalist statuses

**Next steps:**
Establish the changes that need to be made. Read in the sensitive and conservation list for each state.
    a. new - any new species that appear in the state lists but do not have a status in inaturalist (new template)
    b. updates - any changes to information which was added by us previously (user_id = 708886) (update template, action='UPDATE')
    c. removals - any statuses which were added by us previously (user_id = 708886) list which are incorrect (update template, action='REMOVE')
    d. flags - are there any statuses by other users that need to be flagged?

## Prep - common to all states
1. Read in the inaturalist statuses & filter out this state
2. Read in the inaturalist taxa list
3. Read in the state sensitive and conservation list
4. Attempt to match the state statuses to an IUCN equivalent
5. Determine the best placeID to use for this state


### 1. iNaturalist statuses

In [127]:
import pandas as pd
import sys
import os
projectdir = "/Users/oco115/PycharmProjects/authoritative-lists/" # basedir for this gh project
sys.path.append(os.path.abspath(projectdir + "source-code/includes"))
import list_functions as lf

# projectdir = "/Users/new330/IdeaProjects/authoritative-lists/" # basedir for this gh project
sourcedir = projectdir + "source-data/inaturalist-statuses/"
listdir = projectdir + "current-lists/"


# read in the statuses
taxastatus = pd.read_csv(sourcedir + "inat-aust-status-taxa.csv", encoding='UTF-8',na_filter=False,dtype=str) ## Read inaturalist conservation statuses file
taxastatus.head(3)

,id,taxon_id,user_id,place_id,source_id,authority,status,url,description,geoprivacy,...,genus,specificEpithet,infraspecificEpithet,modified,scientificName,taxonRank,references,preferred_common_name,is_active,current_synonymous_taxon_ids
0,166449,38493,1138587,7830,,Flora and Fauna Guarantee Act 1988,CR,,,obscured,...,Eulamprus,kosciuskoi,,2021-03-01T10:35:01Z,Eulamprus kosciuskoi,species,http://reptile-database.reptarium.cz/search.ph...,,,
1,234788,918383,702203,9994,,Atlas of Living Australia,NT,https://bie.ala.org.au/species/https://id.biod...,,,...,Chiloschista,phyllorhiza,,2022-01-08T03:30:36Z,Chiloschista phyllorhiza,species,http://www.catalogueoflife.org/annual-checklis...,,,
2,234789,918383,702203,7308,,Atlas of Living Australia,LC,https://bie.ala.org.au/species/https://id.biod...,,,...,Chiloschista,phyllorhiza,,2022-01-08T03:30:36Z,Chiloschista phyllorhiza,species,http://www.catalogueoflife.org/annual-checklis...,,,


In [128]:
def filter_state_statuses(stateregex: str, urlregex: str):
    authoritydf = taxastatus['authority'].drop_duplicates().sort_values()
    authoritydf = authoritydf[pd.Series(authoritydf).str.contains(stateregex)]
    urldf = taxastatus['url'].drop_duplicates().sort_values()
    urldf = urldf[pd.Series(urldf).str.contains(urlregex)]
    placedisplaydf = taxastatus['place_display_name'].drop_duplicates().sort_values()
    placedisplaydf = placedisplaydf[pd.Series(placedisplaydf).str.contains(stateregex)]
    placedf = taxastatus['place_name'].drop_duplicates().sort_values()
    placedf = placedf[pd.Series(placedf).str.contains(stateregex)]
    # concat all and remove duplicates
    statedf = pd.concat([taxastatus.apply(lambda row: row[taxastatus['place_display_name'].isin(placedisplaydf)]),
                         taxastatus.apply(lambda row: row[taxastatus['place_name'].isin(placedf)]),
                         taxastatus.apply(lambda row: row[taxastatus['url'].isin(urldf)]),
                         taxastatus.apply(
                             lambda row: row[taxastatus['authority'].isin(authoritydf)])]).drop_duplicates()
    return statedf.sort_values(['taxon_id', 'user_id'])


inatstatuses = filter_state_statuses("Qld|QLD|Queensland|QUEENSLAND|QL", ".qld.")
inatstatuses.rename(columns={'id':'status_id','id_y':'taxon_id_y'},inplace=True)
inatstatuses

,status_id,taxon_id,user_id,place_id,source_id,authority,status,url,description,geoprivacy,...,genus,specificEpithet,infraspecificEpithet,modified,scientificName,taxonRank,references,preferred_common_name,is_active,current_synonymous_taxon_ids
532,223098,1003032,3669610,7308,,Queensland Nature Conservation Act 1992,CE,https://apps.des.qld.gov.au/species-search/det...,,obscured,...,Antrophyum,austroqueenslandicum,,2021-10-01T10:37:36Z,Antrophyum austroqueenslandicum,species,,,,
131,169944,1004434,58320,7308,,Queensland Nature Conservation Act 1992,Least concern,https://apps.des.qld.gov.au/species-search/det...,Listed as 'confidential' by the Queensland Dep...,obscured,...,Oberonia,crateriformis,,2021-07-18T09:05:24Z,Oberonia crateriformis,species,http://www.catalogueoflife.org/annual-checklis...,,,
1473,152757,100615,708886,7308,16653,QLD DEHP,endangered,https://data.qld.gov.au/dataset/conservation-s...,,obscured,...,Euastacus,bindal,,2020-05-28T05:44:48Z,Euastacus bindal,species,http://www.iucnredlist.org/apps/redlist/details,,,
1407,152668,100635,708886,7308,16653,QLD DEHP,endangered,https://data.qld.gov.au/dataset/conservation-s...,,obscured,...,Euastacus,jagara,,2020-05-28T05:43:40Z,Euastacus jagara,species,http://www.iucnredlist.org/apps/redlist/details,,,
1267,152461,100638,708886,7308,16653,QLD DEHP,endangered,https://data.qld.gov.au/dataset/conservation-s...,,obscured,...,Euastacus,maidae,,2020-05-28T05:06:01Z,Euastacus maidae,species,http://www.iucnredlist.org/apps/redlist/details,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,262047,977629,3669610,7308,,Queensland Nature Conservation Act 1992,NT,https://www.data.qld.gov.au/dataset/conservati...,,open,...,Melaleuca,formosa,,2022-09-05T10:47:52Z,Melaleuca formosa,species,,,,
2615,161807,993333,702203,7308,,Nature Conservation Act 1992,VU,https://apps.des.qld.gov.au/species-search/det...,,,...,Cupaniopsis,tomentella,,2020-08-30T17:12:00Z,Cupaniopsis tomentella,species,https://eol.org/pages/5629346,,,
2616,161808,993333,702203,6744,,Environment Protection and Biodiversity Conser...,VU,https://apps.des.qld.gov.au/species-search/det...,,,...,Cupaniopsis,tomentella,,2020-08-30T17:12:00Z,Cupaniopsis tomentella,species,https://eol.org/pages/5629346,,,
23,167723,993605,3669610,7308,,QLD DEHP,NT,https://www.data.qld.gov.au/dataset/conservati...,,obscured,...,Acianthus,amplexicaulis,,2021-10-05T08:48:02Z,Acianthus amplexicaulis,species,http://www.catalogueoflife.org/annual-checklis...,,,


### 2. iNaturalist taxonomy

In [129]:
# Output files contain these fields
# Taxon_Name,Status,Authority,IUCN_equivalent,Description,iNaturalist_Place_ID,url,Taxon_Geoprivacy,Username,taxon_id
# so we need to match species from the state lists to the inat taxa to get the taxon_id

import zipfile
url = "https://www.inaturalist.org/taxa/inaturalist-taxonomy.dwca.zip"
filename = url.split("/")[-1]

z=zipfile.ZipFile(sourcedir + filename)

with z.open('taxa.csv') as from_archive:
    inattaxa = pd.read_csv(from_archive,dtype=str)
z.close()
inattaxa.head(3)


,id,taxonID,identifier,parentNameUsageID,kingdom,phylum,class,order,family,genus,specificEpithet,infraspecificEpithet,modified,scientificName,taxonRank,references
0,1,https://www.inaturalist.org/taxa/1,https://www.inaturalist.org/taxa/1,https://www.inaturalist.org/taxa/48460,Animalia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-11-02T06:05:44Z,Animalia,kingdom,http://www.catalogueoflife.org/annual-checklis...
1,2,https://www.inaturalist.org/taxa/2,https://www.inaturalist.org/taxa/2,https://www.inaturalist.org/taxa/1,Animalia,Chordata,NaN,NaN,NaN,NaN,NaN,NaN,2021-11-23T00:40:18Z,Chordata,phylum,http://www.catalogueoflife.org/annual-checklis...
2,3,https://www.inaturalist.org/taxa/3,https://www.inaturalist.org/taxa/3,https://www.inaturalist.org/taxa/355675,Animalia,Chordata,Aves,NaN,NaN,NaN,NaN,NaN,2022-12-27T07:33:16Z,Aves,class,http://www.catalogueoflife.org/annual-checklis...


### 3. State lists

Sensitive list: `geoprivacy` = `obscured`
overrides


In [130]:
sensitivelist = lf.download_ala_list("https://lists.ala.org.au/ws/speciesListItems/dr493?max=10000&includeKVP=true")
sensitivelist = lf.kvp_to_columns(sensitivelist)
sensitivelist['name'] = sensitivelist['name'].str.replace('subsp. ', '', regex=False)
sensitivelist = sensitivelist.rename(columns={'taxonID':'wildnetTaxonID'})
sensitivelist['wildnet_geoprivacy'] = 'obscured'
sensitivelist

,id,name,commonName,scientificName,lsid,dataResourceUid,kvpValues,wildnetTaxonID,scientificNameAuthorship,sourceStatus,Significant,status,Endemicity,E P B C Status,wildnet_geoprivacy
0,4195901,Rhinolophus philippinensis,Greater Large-eared Horseshoe Bat,Rhinolophus philippinensis,ALA_DR652_255,dr493,"[{'key': 'taxonID', 'value': '969'}, {'key': '...",969,"Waterhouse, 1843",E,Y,Endangered,Regional Endemic,Vulnerable,obscured
0,4195892,Chloebia gouldiae,Gouldian Finch,Chloebia gouldiae,https://biodiversity.org.au/afd/taxa/27d27be2-...,dr493,"[{'key': 'taxonID', 'value': '1376'}, {'key': ...",1376,"(Gould, 1844)",E,Y,Endangered,Intranational,Endangered,obscured
0,4195926,Erythrura trichroa,Blue-faced Parrot-finch,Erythrura (Acalanthe) trichroa,https://biodiversity.org.au/afd/taxa/cb632014-...,dr493,"[{'key': 'taxonID', 'value': '1378'}, {'key': ...",1378,"(Kittlitz, 1835)",NT,Y,Near Threatened,Not Endemic to Australia,NaN,obscured
0,4196175,Neochmia phaeton evangelinae,None,Neochmia (Neochmia) phaeton evangelinae,https://biodiversity.org.au/afd/taxa/565b6f37-...,dr493,"[{'key': 'taxonID', 'value': '1370'}, {'key': ...",1370,"(Hombron & Jacquinot, 1841)",E,Y,Endangered,Regional Endemic,Endangered,obscured
0,4195837,Poephila cincta cincta,Black-throated Finch (southern Subspecies),Poephila (Poephila) cincta cincta,https://biodiversity.org.au/afd/taxa/9fe878db-...,dr493,"[{'key': 'taxonID', 'value': '1365'}, {'key': ...",1365,"Gould, 1837",E,Y,Endangered,Intranational,Endangered,obscured
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,4196681,Pneumatopteris costata,None,Pneumatopteris costata,https://id.biodiversity.org.au/node/apni/2887362,dr493,"[{'key': 'taxonID', 'value': '11699'}, {'key':...",11699,(Brack.) Holttum,NT,Y,Near Threatened,Regional Endemic,NaN,obscured
0,4196386,Pneumatopteris pennigera,Pākau,Pneumatopteris pennigera,https://id.biodiversity.org.au/node/apni/2899796,dr493,"[{'key': 'taxonID', 'value': '11700'}, {'key':...",11700,(G.Forst.) Holttum,E,Y,Endangered,Not Endemic to Australia,NaN,obscured
0,4196612,Thelypteris confluens,Swamp Fern,Thelypteris confluens,https://id.biodiversity.org.au/node/apni/2913519,dr493,"[{'key': 'taxonID', 'value': '16042'}, {'key':...",16042,(Thunb.) C.V.Morton,V,Y,Vulnerable,Not Endemic to Australia,NaN,obscured
0,4196389,Macadamia jansenii,Bulburin Nut Tree,Macadamia jansenii,https://id.biodiversity.org.au/node/apni/2904937,dr493,"[{'key': 'taxonID', 'value': '8185'}, {'key': ...",8185,C.L.Gross & P.H.Weston,CR,Y,Critically Endangered,Queensland Endemic,Endangered,obscured


In [131]:
conservationlist = lf.download_ala_list("https://lists.ala.org.au/ws/speciesListItems/dr652?max=10000&includeKVP=true")
conservationlist = lf.kvp_to_columns(conservationlist)
conservationlist['name'] = conservationlist['name'].str.replace('subsp. ', '', regex=False)
conservationlist = conservationlist.rename(columns={'taxonID':'wildnetTaxonID'})
conservationlist['wildnet_geoprivacy'] = conservationlist['Confidential'].apply(lambda x: 'obscured' if x == 'Y' else 'open')
conservationlist

,id,name,commonName,scientificName,lsid,dataResourceUid,kvpValues,wildnetTaxonID,scientificNameAuthorship,sourceStatus,Significant,Confidential,status,Endemicity,E P B C Status,wildnet_geoprivacy
0,4195410,Adelotus brevis,Tusked Frog,Adelotus brevis,https://biodiversity.org.au/afd/taxa/32f57cc7-...,dr652,"[{'key': 'taxonID', 'value': '706'}, {'key': '...",706,"(Günther, 1863)",V,Y,N,Vulnerable,Intranational,NaN,open
0,4195251,Philoria kundagungan,Mountain Frog,Philoria kundagungan,https://biodiversity.org.au/afd/taxa/fcb2e700-...,dr652,"[{'key': 'taxonID', 'value': '687'}, {'key': '...",687,"(Ingram & Corben, 1975)",E,Y,Y,Endangered,Intranational,Endangered,obscured
0,4195545,Crinia tinnula,Wallum Froglet,Crinia tinnula,https://biodiversity.org.au/afd/taxa/a101af4a-...,dr652,"[{'key': 'taxonID', 'value': '686'}, {'key': '...",686,"Straughan & Main, 1966",V,Y,N,Vulnerable,Intranational,NaN,open
0,4193923,Mixophyes fleayi,Fleay's Barred Frog,Mixophyes fleayi,https://biodiversity.org.au/afd/taxa/289e1b7c-...,dr652,"[{'key': 'taxonID', 'value': '675'}, {'key': '...",675,"Corben & Ingram, 1987",E,Y,Y,Endangered,Intranational,Endangered,obscured
0,4195714,Mixophyes iteratus,Giant Barred Frog,Mixophyes iteratus,https://biodiversity.org.au/afd/taxa/9eef5215-...,dr652,"[{'key': 'taxonID', 'value': '676'}, {'key': '...",676,"Straughan, 1968",V,Y,Y,Vulnerable,Intranational,Vulnerable,obscured
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,4195477,Macrozamia viridis,None,Macrozamia viridis,https://id.biodiversity.org.au/node/apni/2917053,dr652,"[{'key': 'taxonID', 'value': '6482'}, {'key': ...",6482,D.L.Jones & P.I.Forst.,E,Y,Y,Endangered,Intranational,NaN,obscured
0,4194878,Alpinia hylandii,None,Alpinia hylandii,https://id.biodiversity.org.au/node/apni/2907514,dr652,"[{'key': 'taxonID', 'value': '8948'}, {'key': ...",8948,R.M.Sm.,NT,Y,N,Near Threatened,Queensland Endemic,NaN,open
0,4194227,Amomum queenslandicum,None,Amomum queenslandicum,https://id.biodiversity.org.au/node/apni/2893545,dr652,"[{'key': 'taxonID', 'value': '8949'}, {'key': ...",8949,R.M.Sm.,V,Y,N,Vulnerable,Queensland Endemic,NaN,open
0,4194621,Globba marantina,Kera Kera,Globba marantina,https://id.biodiversity.org.au/node/apni/2901385,dr652,"[{'key': 'taxonID', 'value': '12434'}, {'key':...",12434,L.,V,Y,N,Vulnerable,Regional Endemic,NaN,open


In [132]:
# join them in a way that works for inat (eg sensitive list, geoprivacy = 'obscured'
statelist = pd.concat([sensitivelist[['wildnetTaxonID', 'name', 'status', 'wildnet_geoprivacy', 'lsid']],
                    conservationlist[['wildnetTaxonID', 'name', 'status', 'wildnet_geoprivacy', 'lsid']]]).drop_duplicates()
statelist

,wildnetTaxonID,name,status,wildnet_geoprivacy,lsid
0,969,Rhinolophus philippinensis,Endangered,obscured,ALA_DR652_255
0,1376,Chloebia gouldiae,Endangered,obscured,https://biodiversity.org.au/afd/taxa/27d27be2-...
0,1378,Erythrura trichroa,Near Threatened,obscured,https://biodiversity.org.au/afd/taxa/cb632014-...
0,1370,Neochmia phaeton evangelinae,Endangered,obscured,https://biodiversity.org.au/afd/taxa/565b6f37-...
0,1365,Poephila cincta cincta,Endangered,obscured,https://biodiversity.org.au/afd/taxa/9fe878db-...
...,...,...,...,...,...
0,6296,Xanthorrhoea sp. (Cape Bedford M.Gandini AQ601...,Special least concern,open,https://id.biodiversity.org.au/node/apni/2906735
0,8948,Alpinia hylandii,Near Threatened,open,https://id.biodiversity.org.au/node/apni/2907514
0,8949,Amomum queenslandicum,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2893545
0,12434,Globba marantina,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2901385


In [133]:
# check for duplicates with conflicting information
statelist.groupby('wildnetTaxonID').filter(lambda x: len(x) > 1)#.sort('size',ascending=False)
#df.groupby('hash').filter(lambda group: len(group) > 1).sort('size', ascending=False)

,wildnetTaxonID,name,status,wildnet_geoprivacy,lsid


### 4. Equivalent IUCN statuses

In [134]:
# iucn_statuses = {'Not Evaluated', 'Data Deficient', 'Least Concern', 'Near Threatened', 'Vulnerable', 'Endangered', 'Critically Endangered', 'Extinct in the Wild','Extinct'}
statelist.groupby(['status'])['status'].count()

status
Critically Endangered    136
Endangered               293
Extinct                    5
Extinct in the wild       26
Least concern            143
Near Threatened          277
Special least concern    908
Vulnerable               600
Name: status, dtype: int64

In [135]:
# iucnStatusMappings = {

iucnStatusMappings = {
    'Least concern':'Least Concern',
    'Special least concern':'Least Concern',
    'Extinct in the wild':'Extinct in the Wild'
}

### 5. Determine best place ID to use

In [136]:
inatstatuses.groupby(['place_id','place_name','place_display_name'])['place_id'].count()
# looks like 7308

place_id  place_name             place_display_name           
                                                                    5
144315    Brisbane City          Brisbane City                      2
153119    South East Queensland  South East Queensland, QL, AU      1
18870     Cairns - Pt B          Cairns - Pt B, QL, AU              1
19232     Yarrabah               Yarrabah, QL, AU                   1
6744      Australia              Australia                         12
7308      Queensland             Queensland, AU                   631
Name: place_id, dtype: int64

## Merge iNaturalist statuses with State lists on scientificName

1. Match - updates, even if the statuses are the same we'll update the links and values anyway
2. No match - statuses to be added (additions)
   1.1 No match and no taxnomy - search for synonyms
   1.2 No match
3. Merge the other direction to see if there are deletes?


In [137]:
# join to see which lists already have a status in inaturalist based on scientificName
mergedstatuses = statelist[['wildnetTaxonID','name','status','wildnet_geoprivacy', 'lsid']].merge(inatstatuses[['status_id','scientificName','taxon_id','user_id','description','iucn','authority','status','geoprivacy','place_id','place_display_name']],how="left",left_on='name',right_on='scientificName',suffixes=(None,'_inat')).sort_values(['name'])
mergedstatuses


,wildnetTaxonID,name,status,wildnet_geoprivacy,lsid,status_id,scientificName,taxon_id,user_id,description,iucn,authority,status_inat,geoprivacy,place_id,place_display_name
1676,33360,Abrodictyum brassii,Special least concern,open,https://id.biodiversity.org.au/node/apni/7401847,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1677,33358,Abrodictyum caudatum,Special least concern,open,https://id.biodiversity.org.au/node/apni/7402200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1678,33359,Abrodictyum obscurum,Special least concern,open,https://id.biodiversity.org.au/node/apni/7402565,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1769,11892,Acacia acrionastes,Near Threatened,open,https://id.biodiversity.org.au/node/apni/2900521,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1770,14898,Acacia ammophila,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2899480,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2333,31680,Zieria scopulus,Critically Endangered,open,https://id.biodiversity.org.au/node/apni/2904113,168010,Zieria scopulus,1097883,58320,,50,Queensland Nature Conservation Act 1992,CR,,7308,"Queensland, AU"
2334,28217,Zieria vagans,Critically Endangered,open,https://id.biodiversity.org.au/node/apni/2900138,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2335,3296,Zieria verrucosa,Vulnerable,open,https://id.biodiversity.org.au/taxon/apni/5136...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2336,41054,Zieria wilhelminae,Critically Endangered,open,ALA_DR652_2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [138]:
# prepare the export fields, common to New template and Update template
# new statuses
# Taxon_Name,Status,Authority,IUCN_equivalent,Description,iNaturalist_Place_ID,url,Taxon_Geoprivacy,Username,taxon_id
# updates
# action,taxon_name,id,taxon_id,status,iucn_equivalent,authority,url,geoprivacy,place_id,username
mergedstatuses['new_authority'] = "Qld Department of Environment and Science"
mergedstatuses['new_description'] = "Listed as Confidential - refer to https://www.data.qld.gov.au/dataset/queensland-confidential-species"
mergedstatuses['new_url'] = mergedstatuses['lsid']
# mergedstatuses['new_url'] = "https://apps.des.qld.gov.au/species-search/details/?id=" + mergedstatuses['wildnetTaxonID'].astype(str)
mergedstatuses.rename(columns={'wildnet_geoprivacy':'new_geoprivacy'},inplace=True)
mergedstatuses['new_place_id'] = '7308'  # Queensland, AU
mergedstatuses['new_username'] = 'peggydnew'
mergedstatuses['new_iucn_equivalent'] = mergedstatuses['status'].str.lower().str.strip().map(iucnStatusMappings).fillna('Vulnerable') # map to dictionary
mergedstatuses['new_status'] = mergedstatuses['status'].fillna('Confidential')
mergedstatuses

,wildnetTaxonID,name,status,new_geoprivacy,lsid,status_id,scientificName,taxon_id,user_id,description,...,geoprivacy,place_id,place_display_name,new_authority,new_description,new_url,new_place_id,new_username,new_iucn_equivalent,new_status
1676,33360,Abrodictyum brassii,Special least concern,open,https://id.biodiversity.org.au/node/apni/7401847,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Qld Department of Environment and Science,Listed as Confidential - refer to https://www....,https://id.biodiversity.org.au/node/apni/7401847,7308,peggydnew,Vulnerable,Special least concern
1677,33358,Abrodictyum caudatum,Special least concern,open,https://id.biodiversity.org.au/node/apni/7402200,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Qld Department of Environment and Science,Listed as Confidential - refer to https://www....,https://id.biodiversity.org.au/node/apni/7402200,7308,peggydnew,Vulnerable,Special least concern
1678,33359,Abrodictyum obscurum,Special least concern,open,https://id.biodiversity.org.au/node/apni/7402565,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Qld Department of Environment and Science,Listed as Confidential - refer to https://www....,https://id.biodiversity.org.au/node/apni/7402565,7308,peggydnew,Vulnerable,Special least concern
1769,11892,Acacia acrionastes,Near Threatened,open,https://id.biodiversity.org.au/node/apni/2900521,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Qld Department of Environment and Science,Listed as Confidential - refer to https://www....,https://id.biodiversity.org.au/node/apni/2900521,7308,peggydnew,Vulnerable,Near Threatened
1770,14898,Acacia ammophila,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2899480,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Qld Department of Environment and Science,Listed as Confidential - refer to https://www....,https://id.biodiversity.org.au/node/apni/2899480,7308,peggydnew,Vulnerable,Vulnerable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2333,31680,Zieria scopulus,Critically Endangered,open,https://id.biodiversity.org.au/node/apni/2904113,168010,Zieria scopulus,1097883,58320,,...,,7308,"Queensland, AU",Qld Department of Environment and Science,Listed as Confidential - refer to https://www....,https://id.biodiversity.org.au/node/apni/2904113,7308,peggydnew,Vulnerable,Critically Endangered
2334,28217,Zieria vagans,Critically Endangered,open,https://id.biodiversity.org.au/node/apni/2900138,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Qld Department of Environment and Science,Listed as Confidential - refer to https://www....,https://id.biodiversity.org.au/node/apni/2900138,7308,peggydnew,Vulnerable,Critically Endangered
2335,3296,Zieria verrucosa,Vulnerable,open,https://id.biodiversity.org.au/taxon/apni/5136...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Qld Department of Environment and Science,Listed as Confidential - refer to https://www....,https://id.biodiversity.org.au/taxon/apni/5136...,7308,peggydnew,Vulnerable,Vulnerable
2336,41054,Zieria wilhelminae,Critically Endangered,open,ALA_DR652_2004,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Qld Department of Environment and Science,Listed as Confidential - refer to https://www....,ALA_DR652_2004,7308,peggydnew,Vulnerable,Critically Endangered


## Updates

In [139]:
# those that need to be updated - we found a status
mergedstatuses[mergedstatuses['status_id'].notnull()][['name','status','status_inat','new_geoprivacy','geoprivacy','authority','user_id']]

,name,status,status_inat,new_geoprivacy,geoprivacy,authority,user_id
1775,Acacia attenuata,Vulnerable,VU,open,,Queensland,702203
1776,Acacia barakulensis,Vulnerable,VU,open,obscured,QLD DEHP,702203
1777,Acacia baueri baueri,Vulnerable,VU,open,,Queensland,702203
1778,Acacia calantha,Near Threatened,NT,open,,Queensland Government,702203
1787,Acacia hockingsii,Vulnerable,VU,open,open,Queensland Nature Conservation Act 1992,3669610
...,...,...,...,...,...,...,...
2318,Zieria bifida,Endangered,EN,open,,Queensland Nature Conservation Act 1992,702203
2321,Zieria collina,Vulnerable,VU,open,open,Nature Conservation Act 1992,702203
2323,Zieria exsul,Critically Endangered,CR,open,,Queensland,702203
2326,Zieria gymnocarpa,Critically Endangered,CR,open,,QLD DEHP,409010


In [140]:
# updates - create the data frame
# action,taxon_name,id,taxon_id,status,iucn_equivalent,authority,url,geoprivacy,place_id,username,description
updates = pd.DataFrame(mergedstatuses[mergedstatuses['status_id'].notnull()])
updates.sort_values('scientificName')
updates['action'] = 'UPDATE'
#updates.loc[:,'action'] = 'UPDATE'
updates = updates[['action','name','status_id','taxon_id','new_status','new_iucn_equivalent','new_authority','new_url','new_geoprivacy','new_place_id','new_username','new_description']]
updates.columns = updates.columns.str.replace("new_", "", regex=True)
updates = updates.rename(columns={'name':'taxon_name',
                                  'status_id':'id'})
updates

,action,taxon_name,id,taxon_id,status,iucn_equivalent,authority,url,geoprivacy,place_id,username,description
1775,UPDATE,Acacia attenuata,159918,898644,Vulnerable,Vulnerable,Qld Department of Environment and Science,https://id.biodiversity.org.au/node/apni/2887463,open,7308,peggydnew,Listed as Confidential - refer to https://www....
1776,UPDATE,Acacia barakulensis,158182,1038687,Vulnerable,Vulnerable,Qld Department of Environment and Science,https://id.biodiversity.org.au/node/apni/2903278,open,7308,peggydnew,Listed as Confidential - refer to https://www....
1777,UPDATE,Acacia baueri baueri,159921,1111700,Vulnerable,Vulnerable,Qld Department of Environment and Science,https://id.biodiversity.org.au/node/apni/2888148,open,7308,peggydnew,Listed as Confidential - refer to https://www....
1778,UPDATE,Acacia calantha,160798,1121183,Near Threatened,Vulnerable,Qld Department of Environment and Science,https://id.biodiversity.org.au/node/apni/2920995,open,7308,peggydnew,Listed as Confidential - refer to https://www....
1787,UPDATE,Acacia hockingsii,264938,1252519,Vulnerable,Vulnerable,Qld Department of Environment and Science,https://id.biodiversity.org.au/node/apni/2897499,open,7308,peggydnew,Listed as Confidential - refer to https://www....
...,...,...,...,...,...,...,...,...,...,...,...,...
2318,UPDATE,Zieria bifida,223195,1125311,Endangered,Vulnerable,Qld Department of Environment and Science,https://id.biodiversity.org.au/node/apni/2895613,open,7308,peggydnew,Listed as Confidential - refer to https://www....
2321,UPDATE,Zieria collina,262040,537414,Vulnerable,Vulnerable,Qld Department of Environment and Science,https://id.biodiversity.org.au/node/apni/2897589,open,7308,peggydnew,Listed as Confidential - refer to https://www....
2323,UPDATE,Zieria exsul,159923,1111558,Critically Endangered,Vulnerable,Qld Department of Environment and Science,https://id.biodiversity.org.au/node/apni/2902255,open,7308,peggydnew,Listed as Confidential - refer to https://www....
2326,UPDATE,Zieria gymnocarpa,169061,1244660,Critically Endangered,Vulnerable,Qld Department of Environment and Science,ALA_DR652_1994,open,7308,peggydnew,Listed as Confidential - refer to https://www....


In [141]:
# investigation - which updates are mine (346), which are those not from me (80 not from me)
#updates[updates['user_id']=='708886'][['scientificName','new_status','status_inat','authority','new_authority','description','new_description','geoprivacy','new_geoprivacy']]
#updates[updates['user_id']!='708886'][['user_id','scientificName','new_status','status_inat','authority','new_authority','description','new_description','geoprivacy','new_geoprivacy']]
# those with different statuses
#updates[updates['new_status'].str.lower().str.strip() != updates['status_inat'].str.lower().str.strip()][['scientificName','new_status','status_inat','authority','new_authority','description','new_description','geoprivacy','new_geoprivacy']]
# users who've updated qld statuses who aren't me
#'https://www.inaturalist.org/users/220795','Steven Kurniawidjaja','neontetraploid','US'
#'https://www.inaturalist.org/users/3669610','Craig Robbins','craig-r','AU'
#'https://www.inaturalist.org/users/527710','James Kameron Mitchell','jameskm','US'
#'https://www.inaturalist.org/users/58320','lwnrngr','lwnrngr','NZ'
#'https://www.inaturalist.org/users/702203','Kitty Maurey','kitty12','CA'
#'https://www.inaturalist.org/users/717122','Miguel de Salas','mftasp','TAS'


## No status in iNaturalist via straight scientificName match
The Qld records that didn't match up to a status in iNaturalist

In [142]:
# to add: those that have no inaturalist status - 532!!
noinatstatus = mergedstatuses[mergedstatuses['status_id'].isnull()]
# try to match the taxon name to something in inaturalist
noinatstatus = noinatstatus.merge(inattaxa, how="left", left_on="name",right_on="scientificName")
noinatstatus

,wildnetTaxonID,name,status,new_geoprivacy,lsid,status_id,scientificName_x,taxon_id,user_id,description,...,class,order,family,genus,specificEpithet,infraspecificEpithet,modified,scientificName_y,taxonRank,references
0,33360,Abrodictyum brassii,Special least concern,open,https://id.biodiversity.org.au/node/apni/7401847,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,33358,Abrodictyum caudatum,Special least concern,open,https://id.biodiversity.org.au/node/apni/7402200,NaN,NaN,NaN,NaN,NaN,...,Polypodiopsida,Hymenophyllales,Hymenophyllaceae,Abrodictyum,caudatum,NaN,2022-06-07T08:14:14Z,Abrodictyum caudatum,species,NaN
2,33359,Abrodictyum obscurum,Special least concern,open,https://id.biodiversity.org.au/node/apni/7402565,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11892,Acacia acrionastes,Near Threatened,open,https://id.biodiversity.org.au/node/apni/2900521,NaN,NaN,NaN,NaN,NaN,...,Magnoliopsida,Fabales,Fabaceae,Acacia,acrionastes,NaN,2022-04-06T22:12:50Z,Acacia acrionastes,species,http://powo.science.kew.org/
4,14898,Acacia ammophila,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2899480,NaN,NaN,NaN,NaN,NaN,...,Magnoliopsida,Fabales,Fabaceae,Acacia,ammophila,NaN,2022-04-07T01:03:28Z,Acacia ammophila,species,http://powo.science.kew.org/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1956,9476,Zieria rimulosa,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2913391,NaN,NaN,NaN,NaN,NaN,...,Magnoliopsida,Sapindales,Rutaceae,Zieria,rimulosa,NaN,2021-07-28T04:47:47Z,Zieria rimulosa,species,https://eol.org/pages/52204449
1957,28217,Zieria vagans,Critically Endangered,open,https://id.biodiversity.org.au/node/apni/2900138,NaN,NaN,NaN,NaN,NaN,...,Magnoliopsida,Sapindales,Rutaceae,Zieria,vagans,NaN,2021-07-28T04:47:46Z,Zieria vagans,species,https://eol.org/pages/52204430
1958,3296,Zieria verrucosa,Vulnerable,open,https://id.biodiversity.org.au/taxon/apni/5136...,NaN,NaN,NaN,NaN,NaN,...,Magnoliopsida,Sapindales,Rutaceae,Zieria,verrucosa,NaN,2021-07-28T04:47:50Z,Zieria verrucosa,species,https://eol.org/pages/49431650
1959,41054,Zieria wilhelminae,Critically Endangered,open,ALA_DR652_2004,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [143]:
noinatstatus[noinatstatus['id'].notna()] # there's no status but there is a matching inat taxon (id is the taxon id)
# note: "Dendrobium" matches to both genus and section

,wildnetTaxonID,name,status,new_geoprivacy,lsid,status_id,scientificName_x,taxon_id,user_id,description,...,class,order,family,genus,specificEpithet,infraspecificEpithet,modified,scientificName_y,taxonRank,references
1,33358,Abrodictyum caudatum,Special least concern,open,https://id.biodiversity.org.au/node/apni/7402200,NaN,NaN,NaN,NaN,NaN,...,Polypodiopsida,Hymenophyllales,Hymenophyllaceae,Abrodictyum,caudatum,NaN,2022-06-07T08:14:14Z,Abrodictyum caudatum,species,NaN
3,11892,Acacia acrionastes,Near Threatened,open,https://id.biodiversity.org.au/node/apni/2900521,NaN,NaN,NaN,NaN,NaN,...,Magnoliopsida,Fabales,Fabaceae,Acacia,acrionastes,NaN,2022-04-06T22:12:50Z,Acacia acrionastes,species,http://powo.science.kew.org/
4,14898,Acacia ammophila,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2899480,NaN,NaN,NaN,NaN,NaN,...,Magnoliopsida,Fabales,Fabaceae,Acacia,ammophila,NaN,2022-04-07T01:03:28Z,Acacia ammophila,species,http://powo.science.kew.org/
5,3304,Acacia arbiana,Near Threatened,open,https://id.biodiversity.org.au/node/apni/2906686,NaN,NaN,NaN,NaN,NaN,...,Magnoliopsida,Fabales,Fabaceae,Acacia,arbiana,NaN,2022-04-06T22:14:00Z,Acacia arbiana,species,http://powo.science.kew.org/
6,31097,Acacia argentina,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2888490,NaN,NaN,NaN,NaN,NaN,...,Magnoliopsida,Fabales,Fabaceae,Acacia,argentina,NaN,2022-04-03T23:48:35Z,Acacia argentina,species,http://powo.science.kew.org/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1955,27791,Zieria obovata,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2917229,NaN,NaN,NaN,NaN,NaN,...,Magnoliopsida,Sapindales,Rutaceae,Zieria,obovata,NaN,2020-02-19T19:36:10Z,Zieria obovata,species,https://eol.org/pages/52204447
1956,9476,Zieria rimulosa,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2913391,NaN,NaN,NaN,NaN,NaN,...,Magnoliopsida,Sapindales,Rutaceae,Zieria,rimulosa,NaN,2021-07-28T04:47:47Z,Zieria rimulosa,species,https://eol.org/pages/52204449
1957,28217,Zieria vagans,Critically Endangered,open,https://id.biodiversity.org.au/node/apni/2900138,NaN,NaN,NaN,NaN,NaN,...,Magnoliopsida,Sapindales,Rutaceae,Zieria,vagans,NaN,2021-07-28T04:47:46Z,Zieria vagans,species,https://eol.org/pages/52204430
1958,3296,Zieria verrucosa,Vulnerable,open,https://id.biodiversity.org.au/taxon/apni/5136...,NaN,NaN,NaN,NaN,NaN,...,Magnoliopsida,Sapindales,Rutaceae,Zieria,verrucosa,NaN,2021-07-28T04:47:50Z,Zieria verrucosa,species,https://eol.org/pages/49431650


In [144]:
# there's no status but there is a matching inat taxon (id is the taxon id)
additions = pd.DataFrame(noinatstatus[noinatstatus['id'].notna()])
additions.sort_values(['name'])
additions['action'] = 'ADD'
additions = additions[['action','name','status_id','id','new_status','new_iucn_equivalent','new_authority','new_url','new_geoprivacy','new_place_id','new_username','new_description']]
additions.columns = additions.columns.str.replace("new_", "", regex=True)
additions = additions.rename(columns={'name':'taxon_name',
                                      'id':'taxon_id',
                                  'status_id':'id'})
additions

,action,taxon_name,id,taxon_id,status,iucn_equivalent,authority,url,geoprivacy,place_id,username,description
1,ADD,Abrodictyum caudatum,NaN,451374,Special least concern,Vulnerable,Qld Department of Environment and Science,https://id.biodiversity.org.au/node/apni/7402200,open,7308,peggydnew,Listed as Confidential - refer to https://www....
3,ADD,Acacia acrionastes,NaN,898576,Near Threatened,Vulnerable,Qld Department of Environment and Science,https://id.biodiversity.org.au/node/apni/2900521,open,7308,peggydnew,Listed as Confidential - refer to https://www....
4,ADD,Acacia ammophila,NaN,898600,Vulnerable,Vulnerable,Qld Department of Environment and Science,https://id.biodiversity.org.au/node/apni/2899480,open,7308,peggydnew,Listed as Confidential - refer to https://www....
5,ADD,Acacia arbiana,NaN,898626,Near Threatened,Vulnerable,Qld Department of Environment and Science,https://id.biodiversity.org.au/node/apni/2906686,open,7308,peggydnew,Listed as Confidential - refer to https://www....
6,ADD,Acacia argentina,NaN,898630,Vulnerable,Vulnerable,Qld Department of Environment and Science,https://id.biodiversity.org.au/node/apni/2888490,open,7308,peggydnew,Listed as Confidential - refer to https://www....
...,...,...,...,...,...,...,...,...,...,...,...,...
1955,ADD,Zieria obovata,NaN,956303,Vulnerable,Vulnerable,Qld Department of Environment and Science,https://id.biodiversity.org.au/node/apni/2917229,open,7308,peggydnew,Listed as Confidential - refer to https://www....
1956,ADD,Zieria rimulosa,NaN,1247621,Vulnerable,Vulnerable,Qld Department of Environment and Science,https://id.biodiversity.org.au/node/apni/2913391,open,7308,peggydnew,Listed as Confidential - refer to https://www....
1957,ADD,Zieria vagans,NaN,1247619,Critically Endangered,Vulnerable,Qld Department of Environment and Science,https://id.biodiversity.org.au/node/apni/2900138,open,7308,peggydnew,Listed as Confidential - refer to https://www....
1958,ADD,Zieria verrucosa,NaN,1247620,Vulnerable,Vulnerable,Qld Department of Environment and Science,https://id.biodiversity.org.au/taxon/apni/5136...,open,7308,peggydnew,Listed as Confidential - refer to https://www....


In [145]:
# write these to the file
pd.concat([updates, additions]).to_csv(sourcedir + "qld.csv", index=False)

In [146]:
# what didnt match to a taxon?
noinatstatus[noinatstatus['id'].isna()]


,wildnetTaxonID,name,status,new_geoprivacy,lsid,status_id,scientificName_x,taxon_id,user_id,description,...,class,order,family,genus,specificEpithet,infraspecificEpithet,modified,scientificName_y,taxonRank,references
0,33360,Abrodictyum brassii,Special least concern,open,https://id.biodiversity.org.au/node/apni/7401847,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,33359,Abrodictyum obscurum,Special least concern,open,https://id.biodiversity.org.au/node/apni/7402565,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,41056,Acacia castorum,Vulnerable,open,ALA_DR652_969,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,40993,Acacia forsteri,Critically Endangered,open,ALA_DR652_974,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,14862,Acacia lauta,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2886854,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1930,6296,Xanthorrhoea sp. (Cape Bedford M.Gandini AQ601...,Special least concern,open,https://id.biodiversity.org.au/node/apni/2906735,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1937,41972,Xylosma craynii,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2903337,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1940,41347,Zealandia pustulata pustulata,Special least concern,open,ALA_DR652_1822,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1942,40091,Zeuxine attenuata,Least concern,obscured,https://id.biodiversity.org.au/taxon/apni/5140...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [147]:
noinatstatus.groupby('status').size()

status
Critically Endangered    105
Endangered               223
Extinct                    5
Extinct in the wild       23
Least concern             92
Near Threatened          219
Special least concern    738
Vulnerable               465
dtype: int64

### are there any that need to be removed?
qld list count: 2517
qld inat statuses count: 653

updates to inat status: 570
additional inat status: 1355
qld statuses we can't find a taxon match for in iNaturalist: 606
total: 2531 (explainable via the various genus/section entries that we matched to in the taxonomy)

inat statuses left over: 653-570=83 that may need checking against the above

In [148]:
# what didnt match to a taxon?
noinatstatus[noinatstatus['id'].isna()]


,wildnetTaxonID,name,status,new_geoprivacy,lsid,status_id,scientificName_x,taxon_id,user_id,description,...,class,order,family,genus,specificEpithet,infraspecificEpithet,modified,scientificName_y,taxonRank,references
0,33360,Abrodictyum brassii,Special least concern,open,https://id.biodiversity.org.au/node/apni/7401847,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,33359,Abrodictyum obscurum,Special least concern,open,https://id.biodiversity.org.au/node/apni/7402565,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,41056,Acacia castorum,Vulnerable,open,ALA_DR652_969,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,40993,Acacia forsteri,Critically Endangered,open,ALA_DR652_974,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,14862,Acacia lauta,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2886854,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1930,6296,Xanthorrhoea sp. (Cape Bedford M.Gandini AQ601...,Special least concern,open,https://id.biodiversity.org.au/node/apni/2906735,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1937,41972,Xylosma craynii,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2903337,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1940,41347,Zealandia pustulata pustulata,Special least concern,open,ALA_DR652_1822,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1942,40091,Zeuxine attenuata,Least concern,obscured,https://id.biodiversity.org.au/taxon/apni/5140...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [149]:
noinatstatus.groupby('status').size()

status
Critically Endangered    105
Endangered               223
Extinct                    5
Extinct in the wild       23
Least concern             92
Near Threatened          219
Special least concern    738
Vulnerable               465
dtype: int64

### are there any that need to be removed?
qld list count: 2517
qld inat statuses count: 653

updates to inat status: 570
additional inat status: 1355
qld statuses we can't find a taxon match for in iNaturalist: 606
total: 2531 (explainable via the various genus/section entries that we matched to in the taxonomy)

inat statuses left over: 653-570=83 that may need checking against the above

In [150]:
# inat statuses that aren't in added or updated
inatstatuses[~inatstatuses['taxon_id'].isin(updates['taxon_id'])]

,status_id,taxon_id,user_id,place_id,source_id,authority,status,url,description,geoprivacy,...,genus,specificEpithet,infraspecificEpithet,modified,scientificName,taxonRank,references,preferred_common_name,is_active,current_synonymous_taxon_ids
102,159922,1019990,702203,7308,,Queensland,VU,https://www.legislation.qld.gov.au/view/html/i...,,,...,Acacia,baueri,,2022-04-06T22:03:46Z,Acacia baueri,species,http://www.catalogueoflife.org/annual-checklis...,,,
437,234929,104980,3669610,7308,,Queensland Nature Conservation Act 1992,VU,https://apps.des.qld.gov.au/species-search/det...,,,...,Maccullochella,peelii,,2022-01-28T04:34:59Z,Maccullochella peelii,species,http://www.fishbase.org,,,
18,167722,1073668,3669610,7308,,Queensland Government,LC,https://www.data.qld.gov.au/dataset/conservati...,"Least concern, locations still considered conf...",,...,Liparis,coelogynoides,,2022-07-12T13:44:01Z,Liparis coelogynoides,species,http://www.catalogueoflife.org/annual-checklis...,,,
308,264555,1074506,3669610,7308,,Queensland Department of Environment and Science,LC,https://apps.des.qld.gov.au/species-search/det...,,open,...,Eucalyptus,chloroclada,,2022-05-30T08:10:19Z,Eucalyptus chloroclada,species,http://www.catalogueoflife.org/annual-checklis...,,,
2614,161805,1091300,702203,7308,,Nature Conservation Act 1992,EN,http://155.187.2.69/cgi-bin/sprat/public/publi...,,,...,,,,,Plectranthus habrophyllus,,,,False,[1278873]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
897,154451,937266,708886,7308,16653,QLD DEHP,endangered,https://data.qld.gov.au/dataset/conservation-s...,,obscured,...,,,,,Caladenia caerulea,,,blue finger-orchid,False,[552476]
837,154918,954413,708886,7308,16653,QLD DEHP,endangered,https://data.qld.gov.au/dataset/conservation-s...,,obscured,...,,,,,Oeceoclades pulchra,,,,False,[1357800]
2815,264331,961868,3669610,7308,,Queensland Nature Conservation Act 1992,LC,https://apps.des.qld.gov.au/species-search/det...,,open,...,Eucalyptus,thozetiana,,2022-05-21T15:39:09Z,Eucalyptus thozetiana,species,http://www.catalogueoflife.org/annual-checklis...,,,
1053,155106,963748,708886,7308,16653,QLD DEHP,endangered,https://data.qld.gov.au/dataset/conservation-s...,,obscured,...,Nervilia,simplex,,2020-02-18T23:50:59Z,Nervilia simplex,species,http://sciencepress.mnhn.fr/fr/periodiques/ada...,,,
